# Environment Setup

In [1]:
import os
import sys

import open3d as o3d

from tutorials.open3d import visualization

# Python version
assert sys.version_info >= (3, 8)
# Open3D version
assert o3d.__version__ >= "0.17.0"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 1. Point Cloud Filtering

In [2]:
# Point cloud file I/O
filename = "data/input/iScan-Pcd-1-1.ply"
base_name, extension = os.path.splitext(os.path.basename(filename))

output_filepath = "data/output"
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)

## Step 1. Down-sampling

In [3]:
def down_sample(input_filename, output_filename):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    # Function uniformly down-samples the point cloud, evenly select 1 point for every k points
    down_sampled_pcd = pcd.uniform_down_sample(every_k_points=10)

    # Save the down-sampled point cloud to an output file
    o3d.t.io.write_point_cloud(output_filename, down_sampled_pcd, write_ascii=False)

In [4]:
# Perform down sampling
down_sampled_filename = os.path.join(output_filepath, base_name + " - downsampled" + extension)
down_sample(filename, down_sampled_filename)

In [5]:
# Visualize down sampled result
visualization.visualization(down_sampled_filename)

## Step 2. Crop the point cloud
This step is done in `CloudCompare` software manually.

## Step 3. Outlier removal

In [6]:
cropped_filename = os.path.join(output_filepath, base_name + " - cropped" + extension)

In [7]:
def outlier_removal(input_filename, output_filename, visualize=False):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    # Statistical outlier removal
    filtered_pcd, mask = pcd.remove_statistical_outliers(nb_neighbors=20, std_ratio=16.0)

    # Visualize and save the outlier into a separate file for comparison
    if visualize:
        inlier = pcd.select_by_mask(mask)
        outlier = pcd.select_by_mask(mask, invert=True)

        inlier.paint_uniform_color([0.7, 0.8, 0.9])  # color in light blue
        outlier.paint_uniform_color([1.0, 0.0, 0.0])  # color in red

        o3d.visualization.draw_geometries([inlier.to_legacy(), outlier.to_legacy()],
                                          window_name="Visualization — {}".format(input_filename),
                                          width=1000, height=800, left=400, top=150)

        # Save the outlier for comparison
        outlier_filename = os.path.join(output_filepath, "outlier.ply")
        o3d.t.io.write_point_cloud(outlier_filename, outlier, write_ascii=False)

    # Save the filtered point cloud to an output file
    o3d.t.io.write_point_cloud(output_filename, filtered_pcd, write_ascii=False)

In [8]:
# Perform outlier removal
filtered_filename = os.path.join(output_filepath, base_name + " - filtered" + extension)
outlier_removal(cropped_filename, filtered_filename, visualize=True)